# LangChain Tutorial

In [1]:
pip install langchain

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 15.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
# os.environ["OPENAI_API_KEY"] = "your_api_key"

### Chat / LLM 모델​

두 가지 유형의 언어 모델이 있습니다

- ChatModel: 입력으로 메시지 리스트를 받고 메시지를 반환하는 기본 모델
- LLM: 입력으로 문자열을 받고 문자열을 반환하는 기본 모델 `2024년 1월 4일에 지원 종료.`

```python
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI  # will be shut off on January 4th, 2024.

chat_model = ChatOpenAI()
llm = OpenAI()

```


### Message

기본 메시지 인터페이스는 BaseMessage에 의해 정의되며, 두 가지 필수 속성이 있습니다

- content: 메시지의 내용입니다. 보통 문자열입니다.
- role: BaseMessage가 오는 엔티티입니다.


LangChain은 다양한 역할을 쉽게 구분할 수 있는 여러 객체를 제공합니다:

- HumanMessage: 사용자/인간으로부터 오는 BaseMessage입니다.
- AIMessage: AI/어시스턴트로부터 오는 BaseMessage입니다.
- SystemMessage: 시스템으로부터 오는 BaseMessage입니다.
- FunctionMessage / ToolMessage: 함수 또는 도구 호출의 결과를 포함하는 BaseMessage입니다.

```python
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

llm.invoke(text)
# >> Feetful of Fun

chat_model.invoke(messages)
# >> AIMessage(content="Socks O'Color")
```

이들 역할 중 어느 것도 적절하지 않다면, `role`을 수동으로 지정할 수 있는 ChatMessage 클래스도 있습니다.

LangChain은 LLM과 챗모델 모두에 공통적으로 사용되는 인터페이스를 제공합니다.
그러나 주어진 언어 모델에 대한 프롬프트를 가장 효과적으로 구성하기 위해서는 두 모델의 차이점을 이해하는 것이 유용합니다.

LLM이나 챗모델을 호출하는 가장 간단한 방법은 `.invoke()`를 사용하는 것입니다.

- ChatModel.invoke: `BaseMessage의 리스트`를 입력으로 받고 `BaseMessage`를 반환합니다.
- LLM.invoke: 문자열을 입력으로 받고 문자열을 반환합니다.

## LangChain으로 LLM 호출하기

In [5]:
pip install langchain-community langchain-core

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
llm = ChatOpenAI()

C:\Users\msh07\AppData\Local\Temp\ipykernel_27392\44520985.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [8]:
llm.invoke("안녕")  # 이렇게 호출하는게 권장되진 않음

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Messages

In [ ]:
from langchain.schema import SystemMessage, HumanMessage

In [ ]:
# 대화 시작
messages = [
    SystemMessage(content="너는 AI 번역 모델이다."),
    HumanMessage(content="'안녕' 이 문장이 영어로 뭐야?"),
]

# 챗 모델을 호출하여 응답을 받습니다.
response = llm.invoke(messages)

In [ ]:
response

In [ ]:
response.content

## Prompt Template

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
human_template = "'{text}' 이 문장을 영어로 번역해줘 "
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
chat_prompt

In [ ]:
chat_prompt.invoke({"text": "안녕"})

In [ ]:
chat_prompt.invoke({"text": "잘가"})

## Chain

Prompt template과 LLM 연결하기

In [ ]:
# 요약 template
human_message_prompt = "'{text}' 여기서 키워드를 뽑아서 콤마로 구분해줘"
human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_message_prompt)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt_template])

In [ ]:
chain = chat_prompt_template | llm

In [ ]:
out = chain.invoke({"text": "LangChain is a framework for developing applications powered by language models."})

In [ ]:
out

### Output Parser

In [ ]:
from langchain.schema import StrOutputParser

In [ ]:
chain_with_output_parser = chat_prompt_template | llm | StrOutputParser()

In [ ]:
out = chain_with_output_parser.invoke({"text": "LangChain is a framework for developing applications powered by language models."})

In [ ]:
out

#### Custom output Parser

In [ ]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

In [ ]:
chain_with_comma_parser = chat_prompt_template | llm | CommaSeparatedListOutputParser()

In [ ]:
out = chain_with_comma_parser.invoke({"text": "LangChain is a framework for developing applications powered by language models."})

In [ ]:
out

In [ ]:
for keyword in out:
    print(keyword)

### Multiple chain

In [ ]:
# Reference: https://techcrunch.com/2023/11/29/a-timeline-of-sam-altmans-firing-from-openai-and-the-fallout/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAABj9TJeKpfEe-qMrcSB1ZKRYrZ9NitZC84VpXuK3PdufAb1F4VMgQEyoGiLa3fQVRg3zA4CPhQmn-LKO2SsxoDiQchz292_G80ZxNOx34WQ_3Nad0xa_LtwILVKTBgSyjavB3uERdh8SdYbuZ0CSuifL21FfXxjD4Ldv3VlKdm0x
text = """
In a dramatic turn of events late Friday, ex-Y Combinator president Sam Altman was fired as CEO of AI startup OpenAI, the company behind viral AI hits like ChatGPT, GPT-4 and DALL-E 3, by OpenAI’s board of directors. Then, the company’s longtime president and co-founder, Greg Brockman, resigned — as did three senior OpenAI researchers. And the fallout continues.

TIP TECHCRUNCH

Do you work at OpenAI and know more about Sam Altman’s departure? Get in touch with TechCrunch.
It’s a fast-moving situation that we’re still trying to get to the bottom of. No doubt more will become clear as time goes on. To make it easier to follow all that’s happened in the meantime, though, we’ve put together a timeline; we’ll do our best to keep it current.

Timeline of Sam Altman’s firing from OpenAI

November 29

Microsoft gains a board observer

Microsoft will gain representation on the new initial board of directors in the form of a non-voting observer, OpenAI announced. It wasn’t immediately clear who this observer might be — only that they won’t have an official vote in board business.

November 21

Sam Altman, OpenAI reach agreement on return as CEO and ‘initial’ new board

In a sudden late announcement, OpenAI revealed that it and Altman “have reached an agreement in principle” for him to return as the company’s CEO. In addition to Altman’s return, its new “initial” board will include former Salesforce chief executive Bret Taylor, former US Secretary of the Treasury Larry Summers and Quora founder Adam D’Angelo.


Altman also posted about the new deal, giving some insight into the roller coaster that we’ve all been riding since his firing was revealed Friday. In his words, his decision to join Microsoft on Sunday “was the best path for me and the team.” Since then, the new board’s composition and Microsoft’s support appear to have been enough to bring him back to the AI company he co-founded.

Altman and board in talks

OpenAI’s board of directors is reportedly in talks with Sam Altman, ex-Y Combinator president and an OpenAI co-founder, to return to OpenAI as CEO as soon as this week. That’s according to Bloomberg, which in a brief — citing sources close to the matter — said that discussions are happening between Quora CEO Adam D’Angelo, one current member of the OpenAI board, and Altman — and possibly other board members as well.

Board tensions boil over

The New York Times reports that, before his ousting, Sam Altman made a move to push out board member Helen Toner because he thought a paper she had co-written was overly critical of OpenAI. That, among other issues, led to OpenAI’s current predicament. Speaking of, The Times indicates that negotiations to hire Altman back continue — but that one major sticking point remaining is “guardrails” meant to improve Altman’s communication with the board.


—

November 20

Altman joins Microsoft

Sam Altman, Greg Brockman and colleagues announce that they’ll join Microsoft to lead a new AI research team. Nadella leaves the door open to other OpenAI staffers, saying that they’ll be given the resources they need should they choose to join.

Sutskever’s mea culpa

Sutskever publishes a post on X suggesting that he regrets his decision to remove Altman and that he’ll do everything in his power to reinstate Altman as CEO.


Employees threaten to resign

Nearly 500 of OpenAI’s roughly 770 employees — including, remarkably, Sutskever — publish a letter saying that they might quit unless the startup’s board resigns and reappoints the ousted Altman. Later Monday, that number climbed to over 650.

Altman and Brockman considering return

As reported by The Verge, Altman’s move to Microsoft isn’t a done deal — and both Altman and Brockman are still open to returning to OpenAI. That is, if the remaining board members who initially fired him step aside.

OpenAI board considers merger

OpenAI’s board of directors approached Dario Amodei, the co-founder and CEO of rival large-language model developer Anthropic, about a potential merger of the two companies, The Information reports. The approach was part of an effort by OpenAI to persuade Amodei to replace Altman as CEO — but Amodei quickly turned down the CEO offer.

—

November 19

Altman to meet at OpenAI HQ

According to The Information, Altman is expected to meet at OpenAI’s San Francisco headquarters as executives at OpenAI push to have him reinstated as CEO. Brockman was invited to join — but it’s unclear whether he’ll take execs up on that invitation.

Board negotiations hit a snag

Bloomberg reports that Lightcap and Murati, among others, are pushing the board to reinstate Altman. But unsurprisingly, the directors are resisting. As of midday Sunday, the board hadn’t resigned out of concern over who could replace them, and were vetting candidates. One possible new addition could be Salesforce co-CEO Bret Taylor.

Altman out, Shear in

Altman won’t be returning as CEO, according to a report in The Information citing an internal memo sent by Sutskever. As the search for a new permanent CEO continues, OpenAI has appointed Emmett Shear, the co-founder of video streaming site Twitch, as interim CEO — replacing Murati.

—

November 18

“Not … in response to malfeasance”

In an internal memo obtained by Axios sent Saturday morning, OpenAI COO Brad Lightcap said yesterday’s announcement “took [the management team] by surprise” and that management had had “multiple conversations with the board to try to better understand the reasons and process behind their decision.” Discussions were ongoing as of Saturday morning, per the memo.

“We can say definitively that the board’s decision was not made in response to malfeasance or anything related to our financial, business, safety, or security/privacy practices,” Lightcap added. “This was a breakdown in communication between Sam and the board … We still share your concerns about how the process has been handled, are working to resolve the situation, and will provide updates as we’re able.”

OpenAI’s funding in jeopardy

The planned sale of OpenAI employee shares that would value the startup at about $86 billion could be in jeopardy. The Information, speaking to three sources formerly with the company, reports that they no longer expect the sale — led by Thrive Capital — to happen, or, if it does, to come with a lesser valuation because of the recent turn of events.

Altman planning new venture

Altman has been telling investors that he’s planning to launch a new venture, according to The Information. Brockman is expected to join the effort — whatever form it takes. (Possibly an AI chip startup.)


Investors pushing for Altman’s return

Investors — furious at the turn of events — are reportedly exerting pressure on OpenAI’s board to reinstate Altman, going so far as to recruit Microsoft. Nadella is said to be sympathetic.

Board agrees to reverse course — in principle

The Verge reports that the board agreed in principle to resign and to allow Altman and Brockman to return. It waffled, however, missing a deadline yesterday by which many OpenAI staffers were set to leave the company. Altman is said to be ambivalent about coming back and asking for “significant” governance changes.

—

November 17

Brockman demoted

Brockman says he got a text from Sutskever shortly after noon on Friday asking for a quick call. After sending a Google Meet link, Brockman was told that he was being removed from the board as chairman “but was vital to the company and would retain his role” as president, and that Altman had been fired.

Altman’s firing publicly announced

OpenAI published a post on its blog announcing the executive shake-up. The company’s management team was aware shortly after.


All-hands meeting

OpenAI held an all-hands meeting Friday afternoon during which Sutskever defended Altman’s ouster. He dismissed suggestions that pushing Altman out amounted to a “hostile takeover,” and claimed that it was necessary to protect OpenAI’s mission of “making AI beneficial to humanity.”

Microsoft releases a statement

Satya Nadella, the CEO of Microsoft, a major investor in — and partner with — OpenAI, published a statement about Altman’s firing:

“As you saw at Microsoft Ignite this week, we’re continuing to rapidly innovate for this era of AI, with over 100 announcements across the full tech stack from AI systems, models and tools in Azure, to Copilot. Most importantly, we’re committed to delivering all of this to our customers while building for the future. We have a long-term agreement with OpenAI with full access to everything we need to deliver on our innovation agenda and an exciting product roadmap; and remain committed to our partnership, and to Mira and the team. Together, we will continue to deliver the meaningful benefits of this technology to the world.”

Brockman quits

Brockman announced his resignation from OpenAI, citing “today’s news.” After sending a memo internally, he published the text on X.


Senior OpenAI researchers resign

Three senior OpenAI researchers resign after Brockman, including the director of research Jakub Pachocki and head of preparedness Aleksander Madry.

—

November 16

Ilya Sutskever schedules call with Altman

According to a post on X (formerly Twitter) from Brockman, Ilya Sutskever, the chief scientist at OpenAI and a co-founder, texted Altman on Thursday evening about scheduling a Friday noon call.


Murati told of Altman’s firing

Brockman alleges that Mira Murati, OpenAI’s CTO and now interim CEO, was informed on Thursday night that Altman would be fired.
"""

In [ ]:
# 요약 template
summarize_human_message_prompt = "'{text}' 이 내용 bullet point 사용해서 3줄 요약해줘"
summarize_human_message_prompt_template = HumanMessagePromptTemplate.from_template(summarize_human_message_prompt)
summarize_chat_prompt_template = ChatPromptTemplate.from_messages([summarize_human_message_prompt])

In [ ]:
# 번역 template
translate_human_message_prompt = "'{text}' 이 내용 한국어로 번역해줘. 번역된 내용만 응답해줘"
translate_human_message_prompt_template = HumanMessagePromptTemplate.from_template(translate_human_message_prompt)
translate_chat_prompt_template = ChatPromptTemplate.from_messages([translate_human_message_prompt_template])

In [ ]:
summarize_chain = summarize_chat_prompt_template | llm | StrOutputParser()

summarize_translate_chain = (
    {"text": summarize_chain}
    | translate_chat_prompt_template
    | llm
    | StrOutputParser() )

In [ ]:
result = summarize_translate_chain.invoke({"text": text})

In [ ]:
print(result)